<a href="https://colab.research.google.com/github/harshalDharpure/Multimodality_Hateful_Meme/blob/main/Bias_weight_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Modified for Hindi datasets Prompthate code

In [ ]:
import torch
import numpy as np
import random
import config
import os
from train import train_for_epoch, WeightedBCELoss  # Import WeightedBCELoss
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer

# Import a function to compute bias weight (you should implement this function)
from utils import compute_bias_weight

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

if __name__ == '__main__':
    opt = config.parse_opt()
    torch.cuda.set_device(opt.CUDA_DEVICE)
    set_seed(opt.SEED)

    # Create tokenizer
    tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

    constructor = 'build_baseline'
    if opt.MODEL == 'pbm':
        from dataset import Multimodal_Data
        import baseline
        train_set = Multimodal_Data(opt, tokenizer, opt.DATASET, 'train', opt.SEED - 1111)
        test_set = Multimodal_Data(opt, tokenizer, opt.DATASET, 'test')
        label_list = [train_set.label_mapping_id[i] for i in train_set.label_mapping_word.keys()]
        model = getattr(baseline, constructor)(opt, label_list).cuda()
    else:
        from roberta_dataset import Roberta_Data
        import roberta_baseline
        train_set = Roberta_Data(opt, tokenizer, opt.DATASET, 'train', opt.SEED - 1111)
        test_set = Roberta_Data(opt, tokenizer, opt.DATASET, 'test')
        model = getattr(roberta_baseline, constructor)(opt).cuda()

    train_loader = DataLoader(train_set, opt.BATCH_SIZE, shuffle=True, num_workers=1)
    test_loader = DataLoader(test_set, opt.BATCH_SIZE, shuffle=False, num_workers=1)

    # Compute bias weight based on your evaluation (implement compute_bias_weight)
    if opt.MODEL == 'pbm':
        label_distribution = train_set.get_label_distribution()
        bias_weight = compute_bias_weight(label_distribution)
    else:
        bias_weight = None

    train_for_epoch(opt, model, train_loader, test_loader, bias_weight)  # Pass bias_weight

    exit(0)
